In [2]:
import httpx

from rasterio.features import bounds as featureBounds

#from ipyleaflet import Map, TileLayer, GeoJSON
from folium import Map, TileLayer, GeoJson

#from shiny import App, ui
#from shinywidgets import render_widget 

In [3]:
titiler_endpoint = "http://localhost:8080"   # Developmentseed Demo endpoint. Please be kind.
url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a/items/S2A_30TVT_20221112_0_L2A"

In [4]:
item = httpx.get(url).json()
print(item)

{'type': 'Feature', 'stac_version': '1.0.0', 'id': 'S2A_30TVT_20221112_0_L2A', 'properties': {'created': '2022-11-14T06:54:49.284Z', 'platform': 'sentinel-2a', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'eo:cloud_cover': 0.005979, 'proj:epsg': 32630, 'mgrs:utm_zone': 30, 'mgrs:latitude_band': 'T', 'mgrs:grid_square': 'VT', 'grid:code': 'MGRS-30TVT', 'view:sun_azimuth': 169.467826196677, 'view:sun_elevation': 24.259740600657594, 's2:degraded_msi_data_percentage': 0, 's2:nodata_pixel_percentage': 0.000226, 's2:saturated_defective_pixel_percentage': 0, 's2:dark_features_percentage': 0, 's2:cloud_shadow_percentage': 0.002296, 's2:vegetation_percentage': 10.348745, 's2:not_vegetated_percentage': 2.478484, 's2:water_percentage': 87.111628, 's2:unclassified_percentage': 0.002548, 's2:medium_proba_clouds_percentage': 0.003716, 's2:high_proba_clouds_percentage': 0.000508, 's2:thin_cirrus_percentage': 0.001755, 's2:snow_ice_percentage': 0.050325, 's2:product_type': 'S2MSI2A', 's2:pro

In [5]:
for it, asset in item["assets"].items():
    print("Name:", it, "| Format:", asset["type"])

Name: aot | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: blue | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: coastal | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: granule_metadata | Format: application/xml
Name: green | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: nir | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: nir08 | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: nir09 | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: red | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: rededge1 | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: rededge2 | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: rededge3 | Format: image/tiff; application=geotiff; profile=cloud-optimized
Name: scl | Format: image/tiff; application=geotiff; profile=cloud-optimized
N

In [6]:
bounds = featureBounds(item)

m = Map(
    tiles="OpenStreetMap",
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=8
)

geo_json = GeoJson(data=item)
geo_json.add_to(m)
m

In [7]:
# Get Tile URL
r = httpx.get(
    f"{titiler_endpoint}/stac/info",
    params = (
        ("url", url),
        # Get info for multiple assets
        ("assets","visual"), ("assets","red"), ("assets","blue"), ("assets","green"),
    )
).json()
print(r)

ConnectError: [Errno 111] Connection refused

In [ ]:
r = httpx.get(
    f"{titiler_endpoint}/stac/WebMercatorQuad/tilejson.json",
    params = {
        "url": url,
        "assets": "visual",
        "minzoom": 8,  # By default titiler will use 0
        "maxzoom": 14, # By default titiler will use 24
    }
).json()

m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=10
)

tiles = TileLayer(
    tiles=r["tiles"][0],
    min_zoom=r["minzoom"],
    max_zoom=r["maxzoom"],
    opacity=1,
    attr="ESA"
)
tiles.add_to(m)
m